## keras实现Convolutional Neural Network

我们和分类一样，同样使用数据集MNIST。不同的是用的层比较多

In [47]:
import numpy as np
np.random.seed(100)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import Adam

In [48]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 1, 28, 28)   # -1 自动计算sample的个数，1代表channel, 28/28 代表长和宽
X_test = X_test.reshape(-1, 1, 28, 28)
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

In [ ]:
X_train.shape

### [构建模型](https://keras.io/layers/convolutional/)

* filter: feature map的数量为32，大小是3*3
* Padding方法是same 基本保持数据的长度和宽带。 因为是第一层所以需要说明输入数据的 shape ，激励选择 relu 函数。代码如下

In [49]:
# 解决如下报错：
# AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'
import keras.backend.tensorflow_backend as tfback
import tensorflow as tf


print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():  

    if tfback._LOCAL_DEVICES is None:  
        devices = tf.config.list_logical_devices()  
        tfback._LOCAL_DEVICES = [x.name for x in devices]  
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]


tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


In [53]:
model = Sequential()
# 第一层卷积(64, 28, 28)
model.add(Convolution2D(batch_input_shape=(32, 1, 28, 28), # 这里59968/60000
    filters=32, kernel_size=3, strides=1, padding='same', data_format='channels_first',activation='relu'))
# 池化层 (32, 14, 14)
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same', data_format='channels_first'))
# 第二层卷积 和 池化
model.add(Convolution2D(filters=64, kernel_size=5, strides=1, padding='same', data_format='channels_first', activation='relu'))
model.add(MaxPooling2D(2, 2, padding='same', data_format='channels_first'))  # (64, 7, 7)

# (64, 7, 7)输出，输入下面的神经网络
model.add(Flatten())  # 展平
model.add(Dense(1024, activation='relu'))  # 第一层神经网络
model.add(Dense(10, activation='softmax')) # 最后输出层，多类别预测使用激活函数： softmax

# 定义优化器
adam = Adam(lr=0.001)
# 编译模型
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### 训练模型

In [54]:
model.fit(X_train, y_train, epochs=1, batch_size=32)  # 32 和model第一层卷积输入一致

Epoch 1/1
60000/60000 [==============================] - 266s 4ms/step - loss: 0.4471 - accuracy: 0.9433


### 评估模型

In [55]:
# Evaluate the model with the metrics we defined earlier
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}\nTest Accuracy: {accuracy}")

10000/10000 [==============================] - 7s 743us/step
Test Loss: 0.08896584861733718
Test Accuracy: 0.9716453552246094
